In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_train

,Id,Label
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0
5,5,1
6,6,0
7,7,1
8,8,0
9,9,0


In [3]:
import pandas as pd
def load_features(col):
    return pd.read_parquet("filtered_features/filtered_feature{}.gzip".format(col))

In [4]:
import pandas as pd

def load_test_features(col):
    return pd.read_parquet("filtered_test_features/filtered_test_feature{}.gzip".format(col))

In [5]:
NUM_OF_COL = 9

In [6]:
SkipCol = {}
# SkipCol[18] = True
SkipCol

{}

In [7]:
y = df_train['Label']
y = y.values
y

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [8]:
df_test = pd.read_csv('sample_solution.csv')


In [9]:
y.shape

(18662,)

In [10]:
import numpy as np
XTrain = np.load('XTrain.npy', allow_pickle=True)
XTrain.shape

(18662, 931)

In [11]:
XTest = np.load('XTest.npy', allow_pickle=True)
XTest.shape

(6051, 931)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(XTrain, y, test_size=0.2, random_state=42)

In [13]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train, y_train)
valid_set = lgb.Dataset(X_valid, y_valid)

In [14]:
param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = 'auc'
bst = lgb.train(param, train_set, 5, valid_sets=valid_set, early_stopping_rounds=5)
bst.best_iteration 

[1]	valid_0's auc: 0.957392
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.965646
[3]	valid_0's auc: 0.971508
[4]	valid_0's auc: 0.97321
[5]	valid_0's auc: 0.974749
Did not meet early stopping. Best iteration is:
[5]	valid_0's auc: 0.974749


5

In [15]:
bst.best_score['valid_0']['auc']

0.9747489235415701

In [20]:
import lightgbm as lgb
from hyperopt import STATUS_OK

ITER = 50
STOP_ROUND = 5

# Create the dataset
def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    params['num_leaves'] = int(params['num_leaves'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    bst = lgb.train(params, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
  
    # Extract the best score
    best_score = bst.best_score['valid_0']['auc']
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

In [23]:
from hyperopt import hp

# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                                [
                                    'gbdt',
                                    'dart',
                                    'goss'
                                ]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'metric': 'auc'
}

space

{'boosting_type': <hyperopt.pyll.base.Apply at 0x1943707ab00>,
 'colsample_bytree': <hyperopt.pyll.base.Apply at 0x194370795c0>,
 'learning_rate': <hyperopt.pyll.base.Apply at 0x1943707ae80>,
 'metric': 'auc',
 'min_child_samples': <hyperopt.pyll.base.Apply at 0x194370791d0>,
 'num_leaves': <hyperopt.pyll.base.Apply at 0x1943707ad30>,
 'reg_alpha': <hyperopt.pyll.base.Apply at 0x19437079320>,
 'reg_lambda': <hyperopt.pyll.base.Apply at 0x19437079470>,
 'subsample_for_bin': <hyperopt.pyll.base.Apply at 0x19437079048>}

In [24]:
from hyperopt import Trials
# Trials object to track progress
bayes_trials = Trials()

In [ ]:
from hyperopt import fmin
from hyperopt import tpe

MAX_EVALS = 500

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)
best

[1]	valid_0's auc: 0.957368                                                                                            
Training until validation scores don't improve for 5 rounds                                                            
[2]	valid_0's auc: 0.964544                                                                                            
[3]	valid_0's auc: 0.968528                                                                                            
[4]	valid_0's auc: 0.971115                                                                                            
[5]	valid_0's auc: 0.972744                                                                                            
[6]	valid_0's auc: 0.974607                                                                                            
[7]	valid_0's auc: 0.97521                                                                                             
[8]	valid_0's auc: 0.976578             

C:\Users\CCS\Anaconda3\lib\site-packages\lightgbm\callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



[2]	valid_0's auc: 0.955431                                                                                            
[3]	valid_0's auc: 0.958402                                                                                            
[4]	valid_0's auc: 0.957473                                                                                            
[5]	valid_0's auc: 0.959932                                                                                            
[6]	valid_0's auc: 0.959507                                                                                            
[7]	valid_0's auc: 0.959683                                                                                            
[8]	valid_0's auc: 0.960286                                                                                            
[9]	valid_0's auc: 0.960921                                                                                            
[10]	valid_0's auc: 0.962313            

In [26]:
best['boosting_type'] = 'dart'
best['num_leaves'] = int(best['num_leaves'])
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['min_child_samples'] = int(best['min_child_samples'])
bestModel = lgb.train(best, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)

C:\Users\CCS\Anaconda3\lib\site-packages\lightgbm\callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


In [27]:
bestModel.save_model('model.txt', num_iteration=bestModel.best_iteration)

In [28]:
bestModel.num_trees()

50

In [29]:
best

{'boosting_type': 'dart',
 'colsample_by_tree': 0.6448607781042294,
 'learning_rate': 0.09463883633283363,
 'min_child_samples': 30,
 'num_leaves': 73,
 'reg_alpha': 0.07802876512824189,
 'reg_lambda': 0.5068838620779224,
 'subsample_for_bin': 80000}

In [30]:
from lightgbm import LGBMClassifier

gb_clf = LGBMClassifier(boosting_type=best['boosting_type'],
                        num_leaves=best['num_leaves'],
                        learning_rate=best['learning_rate'],
                        subsample_for_bin=best['subsample_for_bin'],
                        min_child_samples=best['min_child_samples'],
                        reg_alpha=best['reg_alpha'],
                        reg_lambda=best['reg_lambda'],
                        colsample_bytree=best['colsample_by_tree'])
gb_clf.fit(XTrain, y)

LGBMClassifier(boosting_type='dart', class_weight=None,
               colsample_bytree=0.6448607781042294, importance_type='split',
               learning_rate=0.09463883633283363, max_depth=-1,
               min_child_samples=30, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=73, objective=None,
               random_state=None, reg_alpha=0.07802876512824189,
               reg_lambda=0.5068838620779224, silent=True, subsample=1.0,
               subsample_for_bin=80000, subsample_freq=0)

In [31]:
probs = gb_clf.predict_proba(XTest)
YTest = probs[:, 0]
YTest

array([0.01188247, 0.04145001, 0.98728835, ..., 0.98544271, 0.60948729,
       0.98730684])

In [32]:
np.max(YTest)

0.9873068351742332

In [33]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.011882
1,1,0.041450
2,2,0.987288
3,3,0.087928
4,4,0.912385
5,5,0.939485
6,6,0.985443
7,7,0.815662
8,8,0.012057
9,9,0.026220


In [34]:
df_test.to_csv('test.csv', index=False)

In [ ]:
from lightgbm import LGBMModel
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

clf = LGBMModel()
parameters = {'learning_rate': sp_randFloat(),
                  'subsample'    : sp_randFloat(),
                  'n_estimators' : sp_randInt(100, 1000),
                  'max_depth'    : sp_randInt(4, 10) 
                 }

randm = RandomizedSearchCV(estimator=gb_clf, param_distributions = parameters, 
                           cv = 5, n_iter = 10, n_jobs=-1)
randm.fit(XTrain, y)

# Results from Random Search
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      randm.best_estimator_)

print("\n The best score across ALL searched params:\n",
      randm.best_score_)

print("\n The best parameters across ALL searched params:\n",
      randm.best_params_)

print("\n ========================================================")

gb_clf = randm.best_estimator_